# Importation des packages nécessaires

In [ ]:
import requests
# Importation de la bibliothèque requests pour envoyer des requêtes HTTP et récupérer le contenu des pages web.

from bs4 import BeautifulSoup as bs
# Importation de BeautifulSoup depuis bs4 pour faciliter le traitement et le parsing du HTML.

import os
# Importation du module os pour interagir avec le système de fichiers (création de dossiers, manipulation de chemins).

from openpyxl import Workbook
# Importation de Workbook depuis openpyxl pour créer et manipuler des fichiers Excel.


# Création dossiers pour enregistrer les fichiers

Ces lignes créent des dossiers spécifiques pour organiser les différents types de fichiers téléchargés ou générés par le script.

Variables de dossiers :

- `excel_folder` : Dossier pour stocker les fichiers Excel contenant des données extraites.
- ``image_folder`` : Dossier pour stocker les images téléchargées depuis les pages web.
- ``pdf_folder`` : Dossier pour stocker les fichiers PDF téléchargés.

- ``os.makedirs(path, exist_ok=True)`` :

- ``path`` : Définit le chemin du dossier à créer.
- ``exist_ok=True`` : Permet d'éviter une erreur si le dossier existe déjà. Le script vérifie donc l'existence du dossier avant d'essayer de le créer.

In [ ]:
excel_folder = 'Fichiers_Excel'
image_folder = 'Images'
pdf_folder = 'PDFs'
# Définition des noms des dossiers pour stocker les fichiers Excel, les images et les PDF.

os.makedirs(excel_folder, exist_ok=True)  # Créer le dossier Excel s'il n'existe pas
os.makedirs(image_folder, exist_ok=True)  # Créer le dossier Images s'il n'existe pas
os.makedirs(pdf_folder, exist_ok=True)    # Créer le dossier PDFs s'il n'existe pas
# Création de chaque dossier avec os.makedirs et vérification d'existence (exist_ok=True).


# Nettoyage du texte

La fonction ``clean_text`` prend en entrée une chaîne de texte brute qui peut contenir des caractères spéciaux ou des balises HTML. Elle utilise des méthodes de remplacement et BeautifulSoup pour transformer le texte en une version propre et lisible.

Paramètres :

- ``text`` : Chaîne de caractères à nettoyer, pouvant inclure des balises HTML et des caractères spéciaux.

Retour :

- Renvoie le texte nettoyé sous forme de chaîne de caractères, sans balises HTML, avec des sauts de ligne pour chaque élément de liste <li>.

Processus :

- Remplace les caractères spéciaux ``\xa0`` (espaces insécables) par des espaces standard pour uniformiser le texte.
- Ajoute des sauts de ligne après les balises ``<li>`` pour améliorer la lisibilité de chaque élément de liste dans le texte extrait.
- Utilise BeautifulSoup pour analyser le texte HTML et extraire uniquement le texte brut sans balises.
- Supprime les espaces superflus en début et fin de chaîne.

In [ ]:
def clean_text(text):
    
    
    text = text.replace('\xa0', ' ').strip()
    # Remplace les caractères d'espace insécable ('\xa0') par des espaces standard et supprime les espaces en début et fin de texte.
    
    text = text.replace('<li>', '<li>\n')
    # Ajoute un saut de ligne après chaque balise <li> pour une meilleure lisibilité du texte extrait.

    soup = bs(text, 'html.parser')
    # Utilise BeautifulSoup pour traiter le texte et supprimer toutes les balises HTML restantes.

    cleaned_text = soup.get_text(separator='\n').strip()
    # Extrait uniquement le texte brut, en utilisant des sauts de ligne comme séparateurs entre les éléments,
    # puis supprime les espaces en début et fin de texte.

    return cleaned_text
    # Retourne le texte nettoyé.


# Exportation des informations extraites dans un fichier excel 

La fonction ``export_to_excel`` permet d'exporter une liste de données vers un fichier Excel structuré avec des en-têtes spécifiques. Elle crée un nouveau fichier Excel ou remplace l'ancien s'il existe déjà.

Paramètres :

- ``data`` : Liste de listes contenant les données à exporter vers le fichier Excel. Chaque sous-liste représente une ligne dans la feuille Excel.

- Retour : Aucun retour direct. Le fichier est enregistré dans le répertoire défini sous excel_folder.

Processus :

- Définit le nom du fichier Excel et les en-têtes des colonnes.
- Supprime le fichier existant avec le même nom pour éviter les duplications de données.
- Crée un nouveau classeur Excel, ajoute les en-têtes en première ligne, puis itère sur chaque ligne de data.
- Applique la fonction clean_text pour nettoyer chaque élément de la ligne de données (à l’exception des colonnes "Images" et "PDFs") et ajoute les lignes dans la feuille Excel.
- Sauvegarde le fichier et gère les erreurs en affichant un message.

In [ ]:
def export_to_excel(data):

    
    try:
        excel_filename = os.path.join(excel_folder, 'data_lab_cercle.xlsx')
        headers = ['Catégorie', 'date', 'Thème', 'Preface', 'Titre', 'Texte', 'Lien', 'Liens utiles']
        # Nom du fichier Excel et définition des en-têtes des colonnes.

        # Supprimer l'ancien fichier Excel s'il existe
        if os.path.exists(excel_filename):
            os.remove(excel_filename) 
        # Vérifie si un fichier Excel avec le même nom existe déjà et le supprime pour éviter la duplication.

        wb = Workbook()
        ws_general = wb.active
        ws_general.title = 'Informations générales'
        ws_general.append(headers)  # Ajouter les en-têtes
        # Crée un nouveau classeur Excel et ajoute les en-têtes de colonnes dans la première ligne.

        for line in data:
            cleaned_line = [clean_text(item) if item else 'N/A' for item in line[:-2]]
            # Nettoie chaque élément de la ligne de données avec `clean_text`, sauf les colonnes "Images" et "PDFs".
            ws_general.append(cleaned_line)
            # Ajoute la ligne nettoyée dans la feuille Excel.

        wb.save(excel_filename)
        print(f"Données exportées avec succès vers : {excel_filename}")
        # Sauvegarde le classeur Excel sous le nom spécifié.

    except Exception as e:
        print(f"Une erreur est survenue lors de l'exportation vers Excel : {str(e)}")
        # En cas d'erreur, affiche un message indiquant la nature de l'erreur.


# Télechargement des images

La fonction ``download_images`` télécharge toutes les images disponibles dans une page web et les enregistre localement dans un dossier défini pour les images. Elle utilise BeautifulSoup pour trouver les balises d'image et requests pour récupérer le contenu des images.

Paramètres :

- ``soup`` : Un objet BeautifulSoup représentant la page web analysée.
- ``link`` : L'URL de la page d'où proviennent les images, utilisée pour générer des noms uniques pour les fichiers.

Retour :

- Renvoie une liste contenant les chemins des images téléchargées. Si aucune image n'est téléchargée ou en cas d'erreur, retourne ``['N/A']``.

Processus :

- Récupère toutes les balises`` <img>`` avec un attribut src pour collecter les URLs des images.
- Pour chaque URL d'image, télécharge l'image en utilisant ``requests.get``.
- Génére un nom unique pour chaque image en utilisant une partie de l'URL de la page et un index.
- Sauvegarde chaque image dans le dossier défini, puis ajoute le chemin du fichier téléchargé à une liste.
- Retourne la liste des chemins des images téléchargées ou ``['N/A']`` en cas d'échec.

In [ ]:
def download_images(soup, link):
    
    
    try:
        images = soup.find_all('img')
        image_urls = [img['src'] for img in images if 'src' in img.attrs]
        # Récupère toutes les balises <img> avec un attribut 'src' et compile leurs URLs.

        downloaded_images = []
        # Initialise une liste pour stocker les chemins des images téléchargées.

        for i, img_url in enumerate(image_urls):
            img_data = requests.get(img_url).content
            # Télécharge le contenu de l'image depuis son URL.

            img_name = f"{link.split('/')[-1]}_{i}.jpg"
            img_path = os.path.join(image_folder, img_name)
            # Définit un nom unique pour chaque image en utilisant une partie de l'URL de la page source et un index.
            
            with open(img_path, 'wb') as handler:
                handler.write(img_data)
                # Écrit le contenu de l'image dans un fichier local avec accès en mode écriture binaire.
            
            downloaded_images.append(img_path)
            # Ajoute le chemin de l'image téléchargée à la liste.

        return downloaded_images if downloaded_images else ['N/A']
        # Retourne la liste des images téléchargées, ou 'N/A' si aucune image n'a été téléchargée.

    except Exception as e:
        print(f"Une erreur est survenue lors du téléchargement des images : {str(e)}")
        return ['N/A']
        # En cas d'erreur, affiche un message d'erreur et retourne 'N/A'.


# Téléchargement des pdfs

La fonction ``download_pdfs`` télécharge tous les fichiers PDF présents sur une page web, les sauvegarde localement, puis renvoie les chemins des fichiers enregistrés.

Paramètres :

- ``soup`` : Un objet BeautifulSoup représentant le contenu HTML de la page web.
- ``link`` : L'URL de la page d'où proviennent les fichiers PDF, utilisée pour créer des noms de fichiers uniques.

Retour :

- Renvoie une liste contenant les chemins des fichiers PDF téléchargés. Si aucun PDF n'est téléchargé ou en cas d'erreur, retourne ``['N/A']``.

Processus :

- Trouve toutes les balises ``<div>`` qui contiennent des liens vers des fichiers PDF en ciblant les classes CSS spécifiques.
- Extrait les URLs des fichiers PDF en vérifiant que le lien contient le titre 'Télécharger'.
- Pour chaque URL, télécharge le fichier PDF en utilisant requests.get.
- Génère un nom unique pour chaque PDF en utilisant une partie de l’URL de la page et un index.
- Sauvegarde chaque PDF dans le dossier défini et ajoute le chemin du fichier téléchargé à une liste.
- Retourne la liste des chemins des PDF téléchargés ou ``['N/A']`` en cas d'absence de téléchargement.

In [ ]:
def download_pdfs(soup, link):
    
    
    try:
        pdf_divs = soup.find_all('div', class_=[
            'col-sm-12 col-xl-5- col-xxl-4 action-hover- d-flex flex-column justify-center', 
            'col-sm-3 col-md-6- col-lg-4 col-xl-3 col-xxl-2 action-hover- d-flex flex-column justify-center align-items-center'
        ])
        # Recherche toutes les balises <div> contenant des PDFs dans des sections spécifiques en fonction de leurs classes.

        pdf_urls = [
            div.find('a', href=True)['href']
            for div in pdf_divs
            if div.find('a', href=True) and div.find('a', href=True).get('title') == 'Télécharger'
        ]
        # Récupère les URLs des PDF uniquement dans les balises <a> ayant le titre 'Télécharger'.

        downloaded_pdfs = []
        # Initialise une liste pour stocker les chemins des PDFs téléchargés.

        for i, pdf_url in enumerate(pdf_urls):
            pdf_data = requests.get(pdf_url).content
            # Télécharge le contenu du PDF depuis son URL.

            pdf_name = f"{link.split('/')[-1]}_{i}.pdf"
            pdf_path = os.path.join(pdf_folder, pdf_name)
            # Définit un nom unique pour chaque PDF en utilisant une partie de l'URL de la page source et un index.

            with open(pdf_path, 'wb') as handler:
                handler.write(pdf_data)
                # Écrit le contenu du PDF dans un fichier local avec accès en mode écriture binaire.

            downloaded_pdfs.append(pdf_path)
            # Ajoute le chemin du PDF téléchargé à la liste.

        return downloaded_pdfs if downloaded_pdfs else ['N/A']
        # Retourne la liste des PDFs téléchargés, ou 'N/A' si aucun PDF n'a été téléchargé.

    except Exception as e:
        print(f"Une erreur est survenue lors du téléchargement des PDFs : {str(e)}")
        return ['N/A']
        # En cas d'erreur, affiche un message d'erreur et retourne 'N/A'.


# Extraction des informations depuis une page web et téléchargement des pdfs et images associés

La fonction ``get_info extrait`` des informations spécifiques (catégorie, thème, préface, date, titre, texte, liens utiles) depuis une page web. Elle télécharge également les images et fichiers PDF associés, puis retourne les informations sous forme de liste structurée.

Paramètres :

- `link` : URL de la page web d'où les informations doivent être extraites.

Retour :

- Renvoie une liste contenant les informations extraites et les chemins des images et PDFs téléchargés. En cas d'erreur, retourne une liste avec des valeurs N/A.

Processus :

- Envoie une requête pour récupérer le contenu HTML de la page, puis utilise BeautifulSoup pour analyser ce contenu.
- Extrait la catégorie, le thème, la préface, la date, les titres et le texte de la page. Pour chaque élément, vérifie si une valeur est disponible ; sinon, utilise N/A.
- Recherche des liens utiles dans les balises`` <a>`` ayant pour titre ``"Je me lance"`` ou ``"Accéder à l'outil"`` et les concatène dans une chaîne.
- Appelle download_images et download_pdfs pour télécharger les images et les fichiers PDF présents sur la page, en sauvegardant les chemins d’accès.
- Retourne une liste contenant toutes les informations extraites et les chemins des fichiers téléchargés.

In [ ]:
def get_info(link):
    
    
    try:
        page = requests.get(link)
        soup = bs(page.content, 'html.parser')
        # Envoie une requête HTTP pour obtenir le contenu de la page et utilise BeautifulSoup pour l'analyser.

        # Vérification et extraction des informations avec gestion du 'N/A'
        categorie_element = soup.find('h1', class_='h2')
        categorie = categorie_element.text.strip() if categorie_element else 'N/A'
        # Extrait la catégorie, ou 'N/A' si elle est absente.

        theme_element = soup.find('h1', class_='wiki-title-theme')
        theme = theme_element.text.strip() if theme_element else 'N/A'
        # Extrait le thème de la page.

        preface_element = soup.find('p', class_='lead')
        preface = preface_element.text.strip() if preface_element else 'N/A'
        # Extrait le texte de préface.

        date_element = soup.find('small', class_='app-card-date text-muted')
        date = date_element.text.strip() if date_element else 'N/A'
        # Extrait la date de la page.

        titre_elements = soup.find_all('h2', class_=['wiki-title-def', 'app-post-title'])
        titre = ' / '.join([elem.text.strip() for elem in titre_elements]) if titre_elements else 'N/A'
        # Concatène tous les titres de la page en les séparant par ' / '.

        texte_elements = soup.find_all(['div', 'article'], class_=['card-text', 'kPost-content'])
        texte = ' '.join([elem.get_text(separator='\n').strip() if elem else 'N/A' for elem in texte_elements]) if texte_elements else 'N/A'
        # Concatène tout le texte principal de la page avec des sauts de ligne.

        # Récupérer l'élément <a> spécifique avec title "Je me lance" ou "Accéder à l'outil"
        liens_utiles = []  # Liste pour stocker tous les liens trouvés
        for a_tag in soup.find_all('a', href=True, title=True):
            if a_tag['title'].strip() in ["Je me lance", "Accéder à l'outil"]:
                liens_utiles.append(a_tag['href'])
        # Ajoute les URLs des liens utiles si le titre correspond à "Je me lance" ou "Accéder à l'outil".

        # Si des liens ont été trouvés, les joindre en chaîne, sinon mettre "N/A"
        liens_utiles = ', '.join(liens_utiles) if liens_utiles else 'N/A'
        # Convertit la liste de liens utiles en une chaîne de texte, ou 'N/A' si aucun lien trouvé.

        downloaded_images = download_images(soup, link)
        image_paths = ', '.join(downloaded_images) if downloaded_images else 'N/A'
        # Télécharge les images de la page et les convertit en une chaîne de chemins ou 'N/A'.

        downloaded_pdfs = download_pdfs(soup, link)
        pdf_paths = ', '.join(downloaded_pdfs) if downloaded_pdfs else 'N/A'
        # Télécharge les PDFs de la page et les convertit en une chaîne de chemins ou 'N/A'.

        # Retourner les informations extraites sous forme de liste
        line = [
            categorie,
            date,
            theme,
            preface,
            titre,
            texte,
            link,
            liens_utiles,  # Insertion de liens utiles ici
            image_paths,
            pdf_paths
        ]
        # Structure les informations dans une liste avec l’ordre des éléments requis.

        return line
        # Retourne la liste contenant toutes les informations extraites.

    except Exception as e:
        print(f"Une erreur est survenue lors de l'extraction des informations : {str(e)}")
        return ['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', link, 'N/A', 'N/A', 'N/A']
        # En cas d'erreur, retourne une liste de valeurs 'N/A' pour chaque élément sauf le lien d'origine.


# Extraction des liens d'une section spécifique d'une page web

Ce script extrait des liens de sections spécifiques d'une page web, récupère des informations de chaque lien, puis exporte toutes les données dans un fichier Excel.

Fonction e``xtract_links(section_id)`` :

- Paramètre : ``section_id –`` l'ID de la section contenant les liens à extraire.
- Retour : Une liste contenant les URLs de chaque lien trouvé dans la section.

Processus :

- Utilise BeautifulSoup pour trouver une ``<div>`` avec l’ID spécifié.
- Extrait tous les liens (href) des éléments ``<li>`` de cette section et les stocke dans une liste.

- Bloc ``if __name__ == "__main__":`` :

- Définit l’URL cible et analyse le contenu de la page avec BeautifulSoup.
- Appelle extract_links pour extraire les liens des sections identifiées par c``ollapse-bas-carbone`` et c``ollapse-bim``.
- Affiche les liens extraits pour vérification.
- Combine tous les liens en une liste unique ``all_links``.
- Pour chaque lien, appelle get_info pour extraire les informations détaillées et les stocke dans ``all_data``.
- Exporte les données collectées dans un fichier Excel avec `export_to_excel`.

In [ ]:
def extract_links(section_id):
    
    section = soup.find('div', id=section_id)
    links = []
    # Initialise une liste vide pour stocker les liens extraits.

    if section:
        list_items = section.find_all('li', class_='list-item col-12 col-lg-6 py-2')
        # Recherche tous les éléments <li> dans la section spécifiée, qui contiennent les liens recherchés.
        
        for item in list_items:
            link = item.find('a', href=True)
            if link:
                links.append(link['href'])
                # Ajoute l'attribut 'href' du lien trouvé à la liste des liens si le lien existe.
    
    return links
    # Retourne la liste de tous les liens extraits de la section.


if __name__ == "__main__":
    # Point d'entrée principal du programme pour extraire les liens, obtenir des informations et les exporter dans un fichier Excel.
    
    url = "https://lab.cercle-promodul.inef4.org/themes"
    # URL de la page cible pour récupérer les sections spécifiques de liens.

    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    # Envoie une requête pour obtenir le contenu de la page et utilise BeautifulSoup pour analyser le HTML.

    links_bas_carbone = extract_links('collapse-bas-carbone')
    links_bim = extract_links('collapse-bim')
    # Appelle la fonction extract_links pour obtenir les liens des sections "Bas carbone" et "BIM".

    print("Liens de la section 'Bas carbone':")
    for link in links_bas_carbone:
        print(link)
        # Affiche tous les liens trouvés dans la section "Bas carbone".

    print("\nLiens de la section 'BIM':")
    for link in links_bim:
        print(link)
        # Affiche tous les liens trouvés dans la section "BIM".

    all_links = links_bas_carbone + links_bim
    # Combine tous les liens des sections "Bas carbone" et "BIM" en une seule liste.

    all_data = []
    # Initialise une liste pour stocker toutes les informations extraites des liens.

    for link in all_links:
        info = get_info(link)
        all_data.append(info)
        # Pour chaque lien, extrait les informations et les ajoute à la liste de données.

    export_to_excel(all_data)
    # Exporte toutes les informations extraites dans un fichier Excel en appelant la fonction export_to_excel.
